# SFT 監督式微調

## 什麼是 SFT (Supervised Fine-Tuning)?

Supervised Fine-Tuning (SFT) 是最直接的微調方法,使用標註好的「輸入-輸出」對來訓練模型。

## SFT 的優勢與限制
**優勢:**
- 概念簡單,易於理解和實現
- 資料需求相對明確
- 訓練穩定,收斂快
- 可以快速適應特定任務

**限制:**
- 需要大量高質量標註資料
- 可能過擬合訓練資料
- 對資料質量極為敏感

## 資料格式設計

Alpaca 格式（單輪對話）：

In [ ]:
{
  "instruction": "將以下英文翻譯成中文",
  "input": "The weather is beautiful today.",
  "output": "今天天氣很好。"
}

Chat 格式（多輪對話）：

In [ ]:
{
  "conversations": [
    {"role": "user", "content": "什麼是機器學習？"},
    {"role": "assistant", "content": "機器學習是人工智慧的一個分支..."},
    {"role": "user", "content": "能舉個例子嗎？"},
    {"role": "assistant", "content": "當然！例如垃圾郵件過濾..."}
  ]
}

實際使用時的 Prompt 模板：

In [ ]:
<|system|>你是一個有幫助的 AI 助手</s>
<|user|>什麼是機器學習？</s>
<|assistant|>機器學習是人工智慧的一個分支...</s>
<|user|>能舉個例子嗎？</s>
<|assistant|>

模型學習在 <|assistant|> 標記後生成回應。

## 訓練流程與實作重點

### Step 1：載入模型與 Tokenizer

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# LLaMA 類模型訓練必關
model.config.use_cache = False

# 設定 padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

- 使用 HuggingFace 的 LLaMA 2 7B
    - 需同意 LLaMA 2 license
    - 並登入 HuggingFace（huggingface-cli login）
- LLaMA 預設會開 KV cache，訓練時會造成錯誤或VRAM暴增，所以一般使用者記得關掉

### Step 2：載入並格式化資料

In [ ]:
dataset = load_dataset("json", data_files="train_data.json")

def format_prompt(example):
    prompt = f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""
    return {"text": prompt}

dataset = dataset.map(format_prompt)

**train_data.json 格式長這樣：**

In [ ]:
[
  {
    "instruction": "請解釋什麼是 Transformer",
    "input": "",
    "output": "Transformer 是一種神經網路架構..."
  }
]

### Step 3：Tokenization

In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,   # 超過長度直接切
        max_length=512,
        padding=False,  # 交給 data collator 動態 padding
    )
    # Causal LM：labels = input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["instruction", "input", "output", "text"],
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

### Step 4：訓練參數

In [ ]:
training_args = TrainingArguments(
    output_dir="./sft_output",
    num_train_epochs=3,  # 跑 3 輪資料
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # 有效 batch size = 16
    learning_rate=2e-5,
    bf16=True,
    fp16=False,
    logging_steps=10,
    save_steps=100,  # 每 100 step 存一次
    save_total_limit=2,  # 最多留 2 個 checkpoint
    warmup_steps=100,   # 防止一開始炸掉
    optim="adamw_torch",
    report_to="none",
)

**Batch Size：**
- 小模型（< 7B）：16-32
- 模型（7B-70B）：4-8

**Learning Rate：**
- SFT 通常用較小的 LR：1e-5 到 5e-5
- 比預訓練的 LR（約 3e-4）小 10 倍
- 原因：避免破壞預訓練學到的知識

**LR Scheduler：**
- warmup_ratio = 0.03  # 前 3% 步驟 warmup
- lr_scheduler = "cosine"  # 餘弦衰減

**Epoch 數量：**
- 通常2-5 epochs就足夠
- 過多 epoch 容易 overfitting

### Step 5：Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

### Step 6：開始訓練

In [ ]:
trainer.train()

### Step 7：儲存最終模型

In [ ]:
trainer.save_model("./sft_final_model")

### 完整程式

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from datasets import load_dataset
import torch

# =========================
# 1. 載入模型與 tokenizer
# =========================
model_name = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# LLaMA 類模型訓練必關
model.config.use_cache = False

# 設定 padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# =========================
# 2. 載入並格式化資料
# =========================
dataset = load_dataset("json", data_files="train_data.json")

def format_prompt(example):
    prompt = f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""
    return {"text": prompt}

dataset = dataset.map(format_prompt)

# =========================
# 3. Tokenization
# =========================
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding=False,  # 交給 data collator 動態 padding
    )
    # Causal LM：labels = input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["instruction", "input", "output", "text"],
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# =========================
# 4. 訓練參數
# =========================
training_args = TrainingArguments(
    output_dir="./sft_output",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # 有效 batch size = 16
    learning_rate=2e-5,
    bf16=True,
    fp16=False,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    warmup_steps=100,
    optim="adamw_torch",
    report_to="none",
)

# =========================
# 5. Trainer
# =========================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# =========================
# 6. 開始訓練
# =========================
trainer.train()

# =========================
# 8. 儲存模型
# =========================
trainer.save_model("./sft_final_model")


## 訓練最佳化
### 超參數調優（Ray Tune）

自動搜尋 learning rate、batch size 等參數，找 eval loss 最小 的組合。

**核心概念：**
- objective：一次完整訓練 + 回報指標
- search space：定義要嘗試的超參數範圍
- ASHAScheduler：提早中止表現差的實驗

In [ ]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.air import session

def objective(config):
    """訓練並回報評估指標"""

    training_args = TrainingArguments(
        learning_rate=config["learning_rate"],
        per_device_train_batch_size=config["batch_size"],
        num_train_epochs=config["num_epochs"],
        warmup_ratio=config["warmup_ratio"],

        evaluation_strategy="epoch",
        logging_strategy="steps",
        logging_steps=50,
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[MonitorCallback()],
    )

    trainer.train()
    eval_results = trainer.evaluate()

    session.report({
        "loss": eval_results["eval_loss"]
    })

# 設定搜尋空間
search_space = {
    "learning_rate": tune.loguniform(1e-5, 1e-3),
    "batch_size": tune.choice([2, 4, 8]),
    "num_epochs": tune.choice([2, 3, 5]),
    "warmup_ratio": tune.uniform(0.0, 0.2)
}

# 執行超參數搜尋
analysis = tune.run(
    objective,
    config=search_space,
    num_samples=20,
    scheduler=ASHAScheduler(metric="loss", mode="min"),
    resources_per_trial={
        "cpu": 4,
        "gpu": 1,  # 教學假設 1 GPU
    }
)

best_config = analysis.best_config

### 持續監控與改進
**即時發現：**
- loss 突然暴增
- 梯度爆炸

In [ ]:
class TrainingMonitor:
    """訓練過程監控"""
    def __init__(self):
        self.metrics_history = []
        self.alert_thresholds = {
            "loss_spike": 2.0,  # loss 突然增加
            "gradient_norm": 10.0   # 梯度爆炸
        }

    def log_step(self, step, metrics):
        """記錄每個訓練步驟"""
        if metrics is None:
            return

        self.metrics_history.append({
            "step": step,
            "timestamp": time.time(),
            **metrics
        })

        self.check_alerts(metrics)

    def check_alerts(self, metrics):
        """檢查異常情況"""
        if len(self.metrics_history) < 2:
            return

        prev_loss = self.metrics_history[-2].get("loss")
        curr_loss = metrics.get("loss")

        # 檢測 loss 異常
        if prev_loss is not None and curr_loss is not None:
            if curr_loss > prev_loss * self.alert_thresholds["loss_spike"]:
                print(f"Loss spike: {prev_loss:.4f} → {curr_loss:.4f}")
                # 可以自動降低學習率或回滾
        
        # 檢測梯度異常
        if metrics.get("grad_norm", 0) > self.alert_thresholds["gradient_norm"]:
            print(f"Gradient explosion: {metrics['grad_norm']:.2f}")

# 使用監控器
monitor = TrainingMonitor()

# 在 Trainer callback 中使用
class MonitorCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            monitor.log_step(state.global_step, logs)

## 常見問題排查

### Overfitting（過擬合）：
- 模型過度記憶訓練資料，失去泛化能力
- 症狀：
    - 訓練 loss 持續下降，驗證 loss 上升
    - 對訓練樣本回答完美，但對新問題回答變差
    - 開始重複訓練資料的確切用詞
- 預防方法：
    1. Early stopping
    2. Dropout
    3. weight decay
    4. 增加資料多樣性
    5. 降低學習率或減少 epoch



### Catastrophic Forgetting（災難性遺忘）：
- 微調過程中，模型忘記預訓練學到的知識
- 症狀：
    - 原本能回答的常識問題變得不會
    - 特定領域能力突然下降（如數學推理）

- 預防方法：
    1. 混入預訓練資料：10-20% 的預訓練資料
    2. 使用較小學習率：保留原有權重
    3. 使用 LoRA 等 PEFT 方法
    4. 限制訓練步驟：不要過度訓練

### OOM（Out Of Memory，記憶體不足）：
- GPU / RAM 不足，導致訓練或推理失敗
- 症狀：
    - CUDA out of memory
    - 訓練中途崩潰
    - 輸入稍長就無法推理
- 預防方法：
    - 減小 batch size
    - 縮短序列長度
    - 使用 FP16 / BF16
    - Gradient accumulation / checkpointing
    - 使用 LoRA 等 PEFT 方法

## SFT 的限制與問題
### 幻覺問題（Hallucination）

SFT 模型可能會自信地生成錯誤資訊：
```
使用者：台灣最高的山是哪座？
模型：台灣最高的山是雪山，海拔3886公尺。
     （錯誤，正確答案是玉山，3952公尺）
```
**原因**：
- SFT 只學習「模仿訓練資料的風格」，不保證事實正確性
- 模型傾向生成流暢、自信的回應，即使不確定
- 訓練資料本身可能包含錯誤

**緩解方法**（但無法完全解決）：
- 提高訓練資料品質
- 加入「不確定」的回答示範
- 結合檢索增強（RAG）


### 人類偏好未被明確建模

SFT 假設「訓練資料中的回應都是好的」，但現實更複雜：
```
問題：推薦一部電影

回應A：我推薦《刺激1995》，這是一部經典的劇情片...
回應B：我推薦《刺激1995》，這是 IMDB 評分最高的電影，
        講述獄中友誼與希望的故事，結局催淚...
```

兩個回應都「正確」，但 B 明顯更好（更詳細、更有用）。SFT 無法區分這種品質差異。

### 無法比較「好 vs 更好」

SFT 的目標函數是「最大化正確回應的機率」，而非「讓好的回應機率 > 差的回應機率」。

**舉例**：
```
問題：如何快速入睡？

差的回應：閉上眼睛就能睡著
好的回應：可以嘗試以下方法：1) 睡前一小時避免螢幕 
          2) 保持房間涼爽黑暗 3) 練習深呼吸...
```
SFT 只要兩個回應都出現在訓練資料中，就會同等看待。它不會學到「好的回應更值得生成」。

後續會介紹的 DPO 和 PPO——它們可以透過比較不同回應的好壞，讓模型學會人類偏好。